# Importing libraries and data

In [1]:
import csv
import math
import random
import numpy as np
import pandas as pd
import gdown
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import plotly.express as px
import time
import random
from scipy.stats import beta

In [2]:
file_id = '1EI8FuKQ5Hho_hfFADLWh4wJ2FFSy8vZS'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'purchases_final_PLEASE_TY.csv'
gdown.download(url, output, quiet=False)

file_id = '1PZCA2ytJtFpkPRRmvyNTRzJO26aiJ9Ot'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'q2_customers.csv'
gdown.download(url, output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1EI8FuKQ5Hho_hfFADLWh4wJ2FFSy8vZS
From (redirected): https://drive.google.com/uc?id=1EI8FuKQ5Hho_hfFADLWh4wJ2FFSy8vZS&confirm=t&uuid=eac9ec25-d79b-4a87-bd8c-bf8dc181abe4
To: /content/purchases_final_PLEASE_TY.csv
100%|██████████| 272M/272M [00:06<00:00, 41.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PZCA2ytJtFpkPRRmvyNTRzJO26aiJ9Ot
To: /content/q2_customers.csv
100%|██████████| 927k/927k [00:00<00:00, 117MB/s]


'q2_customers.csv'

In [3]:
customers = pd.read_csv('q2_customers.csv')
purchases = pd.read_csv('purchases_final_PLEASE_TY.csv')

In [4]:
customers.head()

,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,Cart Abandonment Rate,Check Review,Refund rate,Time in Cart,Time on Page,RFM,Pct Discount,Keep Discount,Category
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,0.733693,0,0.243994,7.039021,0.469268,111,0.562500,1,NaN
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,0.709685,1,0.170417,45.151821,3.010121,323,0.458128,1,NaN
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,0.719513,1,0.306721,19.751185,1.316746,322,0.454861,1,NaN
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,0.645122,0,0.118960,13.694138,0.912943,212,0.382022,1,NaN
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,0.832793,0,0.343723,2.544373,0.169625,211,0.380952,1,NaN


In [5]:
purchases.head()

,Order Date,Purchase Price Per Unit,Quantity,Shipping Address State,Title,ASIN/ISBN (Product Code),Survey ResponseID,Category,Prime,Discounts,Express Shipping
0,2018-12-04,7.98,1,NJ,SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory ...,B0143RTB1E,R_01vNIayewjIIKMF,Electronics,1,0.00,1
1,2018-12-22,13.99,1,NJ,Betron BS10 Earphones Wired Headphones in Ear ...,B01MA1MJ6H,R_01vNIayewjIIKMF,Electronics,1,0.00,1
2,2018-12-24,8.99,1,NJ,NaN,B078JZTFN3,R_01vNIayewjIIKMF,NaN,1,0.00,1
3,2018-12-25,10.45,1,NJ,Perfecto Stainless Steel Shaving Bowl. Durable...,B06XWF9HML,R_01vNIayewjIIKMF,Kitchen & Dining,1,0.05,1
4,2018-12-25,10.00,1,NJ,Proraso Shaving Cream for Men,B00837ZOI0,R_01vNIayewjIIKMF,Beauty & Personal Care,1,0.05,1


# Customer Segmentation

## Saving time/urgency (for non-Prime members)

In [6]:
urgency = customers[customers['Express Shipping'] >= 0.5]
urgency = urgency[urgency['Time in Cart'] <= 20]
urgency = urgency[urgency['Prime'] == 0]
len(urgency)

401

In [7]:
len(urgency) / len(customers[customers['Prime'] == 0])


0.232733604178758

about 21% of non-prime users are more urgent (will pay for expedited shipping more often).

In [8]:
urgency['Cat_Urgency'] = 1
urgency['Cat_Urgency'] = urgency['Cat_Urgency'].astype(int)
urgency = urgency[['Survey ResponseID', 'Cat_Urgency']]
customers = pd.merge(customers, urgency, on='Survey ResponseID', how='left')
customers['Cat_Urgency'] = customers['Cat_Urgency'].fillna(0).astype(int)
customers.head()

,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,Cart Abandonment Rate,Check Review,Refund rate,Time in Cart,Time on Page,RFM,Pct Discount,Keep Discount,Category,Cat_Urgency
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,0.733693,0,0.243994,7.039021,0.469268,111,0.562500,1,NaN,0
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,0.709685,1,0.170417,45.151821,3.010121,323,0.458128,1,NaN,0
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,0.719513,1,0.306721,19.751185,1.316746,322,0.454861,1,NaN,1
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,0.645122,0,0.118960,13.694138,0.912943,212,0.382022,1,NaN,0
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,0.832793,0,0.343723,2.544373,0.169625,211,0.380952,1,NaN,0


## Impulse buyer

In [9]:
impulse = customers[customers['Time in Cart'] <= 20]
impulse = impulse[impulse['Time on Page'] <= 2.5]
impulse = impulse[impulse['Check Review'] == 0 ]
impulse = impulse[impulse['Refund rate'] > 0.05]
len(impulse)

1779

In [10]:
len(impulse) / len(customers)

0.36136502132845827

about 36% of customers are impulse buyers.

In [11]:
impulse['Cat_Impulse'] = 1
impulse['Cat_Impulse'] = impulse['Cat_Impulse'].astype(int)
impulse = impulse[['Survey ResponseID', 'Cat_Impulse']]
customers = pd.merge(customers, impulse, on='Survey ResponseID', how='left')
customers['Cat_Impulse'] = customers['Cat_Impulse'].fillna(0).astype(int)
customers.head()

,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,...,Check Review,Refund rate,Time in Cart,Time on Page,RFM,Pct Discount,Keep Discount,Category,Cat_Urgency,Cat_Impulse
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,...,0,0.243994,7.039021,0.469268,111,0.562500,1,NaN,0,1
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,...,1,0.170417,45.151821,3.010121,323,0.458128,1,NaN,0,0
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,...,1,0.306721,19.751185,1.316746,322,0.454861,1,NaN,1,0
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,...,0,0.118960,13.694138,0.912943,212,0.382022,1,NaN,0,1
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,...,0,0.343723,2.544373,0.169625,211,0.380952,1,NaN,0,1


## Indecisive/cautious

In [12]:
indecisive = customers[customers['Cart Abandonment Rate'] >= 0.75]
indecisive = indecisive[indecisive['Check Review'] == 1]
indecisive = indecisive[indecisive['Time on Page'] >= 7]
indecisive = indecisive[indecisive['Time in Cart'] >= 30]
len(indecisive)

623

In [13]:
len(indecisive) / len(customers)

0.12654885232581758

about 13% of customers are indecisive buyers

In [14]:
indecisive['Cat_Indecisive'] = 1
indecisive['Cat_Indecisive'] = indecisive['Cat_Indecisive'].astype(int)
indecisive = indecisive[['Survey ResponseID', 'Cat_Indecisive']]
customers = pd.merge(customers, indecisive, on='Survey ResponseID', how='left')
customers['Cat_Indecisive'] = customers['Cat_Indecisive'].fillna(0).astype(int)
customers.head()

,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,...,Refund rate,Time in Cart,Time on Page,RFM,Pct Discount,Keep Discount,Category,Cat_Urgency,Cat_Impulse,Cat_Indecisive
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.243994,7.039021,0.469268,111,0.562500,1,NaN,0,1,0
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,...,0.170417,45.151821,3.010121,323,0.458128,1,NaN,0,0,0
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,...,0.306721,19.751185,1.316746,322,0.454861,1,NaN,1,0,0
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,...,0.118960,13.694138,0.912943,212,0.382022,1,NaN,0,1,0
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,...,0.343723,2.544373,0.169625,211,0.380952,1,NaN,0,1,0


## Category

In [15]:
cat = customers[customers['Category'].notnull()]
len(cat)

406

In [16]:
len(indecisive) / len(customers)

0.12654885232581758

about 13% of customers have a frequently purchased category that they stick to when ordering

In [17]:
cat['Cat_Category'] = 1
cat['Cat_Category'] = cat['Cat_Category'].astype(int)
cat = cat[['Survey ResponseID', 'Cat_Category']]
customers = pd.merge(customers, cat, on='Survey ResponseID', how='left')
customers['Cat_Category'] = customers['Cat_Category'].fillna(0).astype(int)
customers.head()

<ipython-input-17-42e965a28865>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat['Cat_Category'] = 1
<ipython-input-17-42e965a28865>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat['Cat_Category'] = cat['Cat_Category'].astype(int)


,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,...,Time in Cart,Time on Page,RFM,Pct Discount,Keep Discount,Category,Cat_Urgency,Cat_Impulse,Cat_Indecisive,Cat_Category
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,...,7.039021,0.469268,111,0.562500,1,NaN,0,1,0,0
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,...,45.151821,3.010121,323,0.458128,1,NaN,0,0,0,0
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,...,19.751185,1.316746,322,0.454861,1,NaN,1,0,0,0
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,...,13.694138,0.912943,212,0.382022,1,NaN,0,1,0,0
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,...,2.544373,0.169625,211,0.380952,1,NaN,0,1,0,0


## Price sensitive

In [18]:
discount = customers[customers['Keep Discount'] == 1]
discount = customers[customers['Pct Discount'] >= 0.4]
len(discount)

3460

In [19]:
len(discount) / len(customers)

0.7028234816169002

In [20]:
discount['Cat_Discount'] = 1
discount['Cat_Discount'] = discount['Cat_Discount'].astype(int)
discount = discount[['Survey ResponseID', 'Cat_Discount']]
customers = pd.merge(customers, discount, on='Survey ResponseID', how='left')
customers['Cat_Discount'] = customers['Cat_Discount'].fillna(0).astype(int)
customers.head()

<ipython-input-20-9f3e8be7b132>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  discount['Cat_Discount'] = 1
<ipython-input-20-9f3e8be7b132>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  discount['Cat_Discount'] = discount['Cat_Discount'].astype(int)


,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,...,Time on Page,RFM,Pct Discount,Keep Discount,Category,Cat_Urgency,Cat_Impulse,Cat_Indecisive,Cat_Category,Cat_Discount
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.469268,111,0.562500,1,NaN,0,1,0,0,1
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,...,3.010121,323,0.458128,1,NaN,0,0,0,0,1
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,...,1.316746,322,0.454861,1,NaN,1,0,0,0,1
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,...,0.912943,212,0.382022,1,NaN,0,1,0,0,0
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,...,0.169625,211,0.380952,1,NaN,0,1,0,0,0


In [21]:
customers = customers.drop(columns = ['Keep Discount'])

## Marketing Engagement

In [22]:
cols_to_check = ['Email CTC', 'Display Ad CTC', 'Video Ad CTC', 'Search Engine Ad CTC', 'Social Media Ad CTC']
filtered_cust = customers[customers[cols_to_check].gt(0).any(axis=1)]
len(filtered_cust)

4307

In [23]:
len(filtered_cust) / len(customers)

0.8748730448913264

about 88% of customers were engaged by any one of the marketing ads

In [24]:
filtered_cust['Cat_Engagement'] = 1
filtered_cust['Cat_Engagement'] = filtered_cust['Cat_Engagement'].astype(int)
filtered_cust = filtered_cust[['Survey ResponseID', 'Cat_Engagement']]
customers = pd.merge(customers, filtered_cust, on='Survey ResponseID', how='left')
customers['Cat_Engagement'] = customers['Cat_Engagement'].fillna(0).astype(int)
customers.head()

<ipython-input-24-a159f1190ae9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cust['Cat_Engagement'] = 1
<ipython-input-24-a159f1190ae9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cust['Cat_Engagement'] = filtered_cust['Cat_Engagement'].astype(int)


,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,...,Time on Page,RFM,Pct Discount,Category,Cat_Urgency,Cat_Impulse,Cat_Indecisive,Cat_Category,Cat_Discount,Cat_Engagement
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.469268,111,0.562500,NaN,0,1,0,0,1,1
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,...,3.010121,323,0.458128,NaN,0,0,0,0,1,1
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,...,1.316746,322,0.454861,NaN,1,0,0,0,1,1
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,...,0.912943,212,0.382022,NaN,0,1,0,0,0,1
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,...,0.169625,211,0.380952,NaN,0,1,0,0,0,1


## High Value Customers

In [25]:
hvc = customers.copy()
hvc[['R', 'F', 'M']] = hvc['RFM'].astype(str).apply(lambda x: pd.Series(list(x)))
hvc[['R', 'F', 'M']] = hvc[['R', 'F', 'M']].astype(int)
hvc.head()

,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,...,Category,Cat_Urgency,Cat_Impulse,Cat_Indecisive,Cat_Category,Cat_Discount,Cat_Engagement,R,F,M
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,...,NaN,0,1,0,0,1,1,1,1,1
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,...,NaN,0,0,0,0,1,1,3,2,3
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,...,NaN,1,0,0,0,1,1,3,2,2
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,...,NaN,0,1,0,0,0,1,2,1,2
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,...,NaN,0,1,0,0,0,1,2,1,1


In [26]:
hvc_f = hvc[hvc['F'] == 3]
len(hvc_f) / len(customers)

0.2506601665651026

about 25% of customers are high value (frequent) customers

In [27]:
hvc_f['Cat_High_Value_F'] = 1
hvc_f['Cat_High_Value_F'] = hvc_f['Cat_High_Value_F'].astype(int)
hvc_f = hvc_f[['Survey ResponseID', 'Cat_High_Value_F']]
customers = pd.merge(customers, hvc_f, on='Survey ResponseID', how='left')
customers['Cat_High_Value_F'] = customers['Cat_High_Value_F'].fillna(0).astype(int)
customers.head()

<ipython-input-27-f8d94d414ed1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvc_f['Cat_High_Value_F'] = 1
<ipython-input-27-f8d94d414ed1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvc_f['Cat_High_Value_F'] = hvc_f['Cat_High_Value_F'].astype(int)


,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,...,RFM,Pct Discount,Category,Cat_Urgency,Cat_Impulse,Cat_Indecisive,Cat_Category,Cat_Discount,Cat_Engagement,Cat_High_Value_F
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,...,111,0.562500,NaN,0,1,0,0,1,1,0
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,...,323,0.458128,NaN,0,0,0,0,1,1,0
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,...,322,0.454861,NaN,1,0,0,0,1,1,0
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,...,212,0.382022,NaN,0,1,0,0,0,1,0
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,...,211,0.380952,NaN,0,1,0,0,0,1,0


In [28]:
hvc_m = hvc[hvc['M'] == 3]
len(hvc_m) / len(customers)

0.2500507820434694

about 25% of customers are high value (high spending) customers

In [29]:
hvc_m['Cat_High_Value_M'] = 1
hvc_m['Cat_High_Value_M'] = hvc_m['Cat_High_Value_M'].astype(int)
hvc_m = hvc_m[['Survey ResponseID', 'Cat_High_Value_M']]
customers = pd.merge(customers, hvc_m, on='Survey ResponseID', how='left')
customers['Cat_High_Value_M'] = customers['Cat_High_Value_M'].fillna(0).astype(int)
customers.head()

<ipython-input-29-6c6b74739680>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvc_m['Cat_High_Value_M'] = 1
<ipython-input-29-6c6b74739680>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvc_m['Cat_High_Value_M'] = hvc_m['Cat_High_Value_M'].astype(int)


,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,...,Pct Discount,Category,Cat_Urgency,Cat_Impulse,Cat_Indecisive,Cat_Category,Cat_Discount,Cat_Engagement,Cat_High_Value_F,Cat_High_Value_M
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.562500,NaN,0,1,0,0,1,1,0,0
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,...,0.458128,NaN,0,0,0,0,1,1,0,1
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,...,0.454861,NaN,1,0,0,0,1,1,0,0
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,...,0.382022,NaN,0,1,0,0,0,1,0,0
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,...,0.380952,NaN,0,1,0,0,0,1,0,0


In [30]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4923 entries, 0 to 4922
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Survey ResponseID      4923 non-null   object 
 1   Prime                  4923 non-null   int64  
 2   Free Trial             4923 non-null   int64  
 3   Email CTC              4923 non-null   float64
 4   Display Ad CTC         4923 non-null   float64
 5   Video Ad CTC           4923 non-null   float64
 6   Search Engine Ad CTC   4923 non-null   float64
 7   Social Media Ad CTC    4923 non-null   float64
 8   Average CTC            4923 non-null   float64
 9   Express Shipping       4896 non-null   float64
 10  Cart Abandonment Rate  4923 non-null   float64
 11  Check Review           4923 non-null   int64  
 12  Refund rate            4923 non-null   float64
 13  Time in Cart           4923 non-null   float64
 14  Time on Page           4923 non-null   float64
 15  RFM 

## Analysis

In [31]:
customers.head()

,Survey ResponseID,Prime,Free Trial,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,...,Pct Discount,Category,Cat_Urgency,Cat_Impulse,Cat_Indecisive,Cat_Category,Cat_Discount,Cat_Engagement,Cat_High_Value_F,Cat_High_Value_M
0,R_03aEbghUILs9NxD,1,1,0.000000,0.030724,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.562500,NaN,0,1,0,0,1,1,0,0
1,R_06RZP9pS7kONINr,1,1,0.224220,0.000000,0.146967,0.000000,0.198426,0.113970,1.000000,...,0.458128,NaN,0,0,0,0,1,1,0,1
2,R_085qq7w0pkhowox,0,1,0.000000,0.000000,0.000000,0.000000,0.012424,0.000000,0.557377,...,0.454861,NaN,1,0,0,0,1,1,0,0
3,R_08uYA7fb4unHGkF,1,1,0.082589,0.000000,0.000000,0.126268,0.020301,0.045888,1.000000,...,0.382022,NaN,0,1,0,0,0,1,0,0
4,R_0BrMxYYMHVMOBEJ,0,1,0.000000,0.016515,0.000000,0.220622,0.011220,0.049910,0.400000,...,0.380952,NaN,0,1,0,0,0,1,0,0


In [32]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4923 entries, 0 to 4922
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Survey ResponseID      4923 non-null   object 
 1   Prime                  4923 non-null   int64  
 2   Free Trial             4923 non-null   int64  
 3   Email CTC              4923 non-null   float64
 4   Display Ad CTC         4923 non-null   float64
 5   Video Ad CTC           4923 non-null   float64
 6   Search Engine Ad CTC   4923 non-null   float64
 7   Social Media Ad CTC    4923 non-null   float64
 8   Average CTC            4923 non-null   float64
 9   Express Shipping       4896 non-null   float64
 10  Cart Abandonment Rate  4923 non-null   float64
 11  Check Review           4923 non-null   int64  
 12  Refund rate            4923 non-null   float64
 13  Time in Cart           4923 non-null   float64
 14  Time on Page           4923 non-null   float64
 15  RFM 

In [37]:
label_count = customers.iloc[:, 18:26].sum(axis=0)
print(label_count)

Cat_Urgency          401
Cat_Impulse         1779
Cat_Indecisive       623
Cat_Category         406
Cat_Discount        3460
Cat_Engagement      4307
Cat_High_Value_F    1234
Cat_High_Value_M    1231
dtype: int64


In [38]:
label_percentage = (label_count / len(customers)) * 100
print(label_percentage)

Cat_Urgency          8.145440
Cat_Impulse         36.136502
Cat_Indecisive      12.654885
Cat_Category         8.247004
Cat_Discount        70.282348
Cat_Engagement      87.487304
Cat_High_Value_F    25.066017
Cat_High_Value_M    25.005078
dtype: float64


In [39]:
impulse_engaged = customers[(customers['Cat_Engagement'] == 1) & (customers['Cat_Impulse'] == 1)].shape[0]
impulse_engaged / 1779 * 100

87.85834738617201

In [40]:
discount_indecisive = customers[(customers['Cat_Indecisive'] == 1) & (customers['Cat_Discount'] == 1)].shape[0]
discount_indecisive / 3460 * 100

11.734104046242773

In [47]:
highv_f_prime = customers[(customers['Cat_High_Value_F'] == 1) & (customers['Prime'] == 1)].shape[0]
highv_m_prime = customers[(customers['Cat_High_Value_M'] == 1) & (customers['Prime'] == 1)].shape[0]
(highv_m_prime + highv_f_prime) / (1234 + 1231) * 100

92.0892494929006

# RFM

In [41]:
def rfm(purchases,customers):
  customers['RFM'] = customers['RFM'].astype(str)
  customers['Churn Risk'] = customers['RFM'].str[0].apply(lambda x: "Active" if x == "3" else ("At risk" if x == "2" else "Churned"))
  return(customers)
print(customers.columns)

Index(['Survey ResponseID', 'Prime', 'Free Trial', 'Email CTC',
       'Display Ad CTC', 'Video Ad CTC', 'Search Engine Ad CTC',
       'Social Media Ad CTC', 'Average CTC', 'Express Shipping',
       'Cart Abandonment Rate', 'Check Review', 'Refund rate', 'Time in Cart',
       'Time on Page', 'RFM', 'Pct Discount', 'Category', 'Cat_Urgency',
       'Cat_Impulse', 'Cat_Indecisive', 'Cat_Category', 'Cat_Discount',
       'Cat_Engagement', 'Cat_High_Value_F', 'Cat_High_Value_M'],
      dtype='object')


In [42]:
#urgency
customers_urgency = customers[customers['Cat_Urgency'] == 1]
trans_urgency = pd.merge(customers_urgency, purchases, on='Survey ResponseID', how='left')
rfm_urgency = rfm(trans_urgency, customers_urgency)

#impulse
customers_impulse = customers[customers['Cat_Impulse'] == 1]
trans_impulse = pd.merge(customers_impulse, purchases, on='Survey ResponseID', how='left')
rfm_impulse = rfm(trans_impulse, customers_impulse)

#indecisive
customers_indecisive = customers[customers['Cat_Indecisive'] == 1]
trans_indecisive = pd.merge(customers_indecisive, purchases, on='Survey ResponseID', how='left')
rfm_indecisive = rfm(trans_indecisive, customers_indecisive)

#category
customers_category = customers[customers["Cat_Category"] == 1]
trans_category = pd.merge(customers_category, purchases, on='Survey ResponseID', how='left')
rfm_category = rfm(trans_category, customers_category)

#engagement
customers_engagement = customers[customers["Cat_Engagement"] == 1]
trans_engagement = pd.merge(customers_engagement, purchases, on='Survey ResponseID', how='left')
rfm_engagement = rfm(trans_engagement, customers_engagement)

#high value F
customers_highvalf = customers[customers['Cat_High_Value_F'] == 1]
trans_highvalf = pd.merge(customers_highvalf, purchases, on='Survey ResponseID', how='left')
rfm_highvalf = rfm(trans_highvalf, customers_highvalf)

#high value M
customers_highvalm = customers[customers['Cat_High_Value_M'] == 1]
trans_highvalm = pd.merge(customers_highvalm, purchases, on='Survey ResponseID', how='left')
rfm_highvalm = rfm(trans_highvalm, customers_highvalm)

#discount
customers_discount = customers[customers["Cat_Discount"] == 1]
trans_discount = pd.merge(customers_discount, purchases, on='Survey ResponseID', how='left')
rfm_discount = rfm(trans_discount, customers_discount)

churn_segments_data = {
    'Segment': [],
    'Active': [],
    'At risk': [],
    'Churned': [],
    'Size': []
}

churn_segments = pd.DataFrame(churn_segments_data)

churn_segments.loc[len(churn_segments)] = ["Urgency", rfm_urgency[rfm_urgency['Churn Risk'] == "Active"].shape[0]/rfm_urgency.shape[0],
                                           rfm_urgency[rfm_urgency['Churn Risk'] == "At risk"].shape[0]/rfm_urgency.shape[0],
                                           rfm_urgency[rfm_urgency['Churn Risk'] == "Churned"].shape[0]/rfm_urgency.shape[0],
                                           rfm_urgency.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Impulse", rfm_impulse[rfm_impulse['Churn Risk'] == "Active"].shape[0]/rfm_impulse.shape[0],
                                           rfm_impulse[rfm_impulse['Churn Risk'] == "At risk"].shape[0]/rfm_impulse.shape[0],
                                           rfm_impulse[rfm_impulse['Churn Risk'] == "Churned"].shape[0]/rfm_impulse.shape[0],
                                           rfm_impulse.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Indecisive", rfm_indecisive[rfm_indecisive['Churn Risk'] == "Active"].shape[0]/rfm_indecisive.shape[0],
                                           rfm_indecisive[rfm_indecisive['Churn Risk'] == "At risk"].shape[0]/rfm_indecisive.shape[0],
                                           rfm_indecisive[rfm_indecisive['Churn Risk'] == "Churned"].shape[0]/rfm_indecisive.shape[0],
                                           rfm_indecisive.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Category", rfm_category[rfm_category['Churn Risk'] == "Active"].shape[0]/rfm_category.shape[0],
                                           rfm_category[rfm_category['Churn Risk'] == "At risk"].shape[0]/rfm_category.shape[0],
                                           rfm_category[rfm_category['Churn Risk'] == "Churned"].shape[0]/rfm_category.shape[0],
                                           rfm_category.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Engagement", rfm_engagement[rfm_engagement['Churn Risk'] == "Active"].shape[0]/rfm_engagement.shape[0],
                                           rfm_engagement[rfm_engagement['Churn Risk'] == "At risk"].shape[0]/rfm_engagement.shape[0],
                                           rfm_engagement[rfm_engagement['Churn Risk'] == "Churned"].shape[0]/rfm_engagement.shape[0],
                                           rfm_engagement.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["High Value Frequency", rfm_highvalf[rfm_highvalf['Churn Risk'] == "Active"].shape[0]/rfm_highvalf.shape[0],
                                           rfm_highvalf[rfm_highvalf['Churn Risk'] == "At risk"].shape[0]/rfm_highvalf.shape[0],
                                           rfm_highvalf[rfm_highvalf['Churn Risk'] == "Churned"].shape[0]/rfm_highvalf.shape[0],
                                           rfm_highvalf.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["High Value Monetary", rfm_highvalm[rfm_highvalm['Churn Risk'] == "Active"].shape[0]/rfm_highvalm.shape[0],
                                           rfm_highvalm[rfm_highvalm['Churn Risk'] == "At risk"].shape[0]/rfm_highvalm.shape[0],
                                           rfm_highvalm[rfm_highvalm['Churn Risk'] == "Churned"].shape[0]/rfm_highvalm.shape[0],
                                           rfm_highvalm.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Discount", rfm_discount[rfm_discount['Churn Risk'] == "Active"].shape[0]/rfm_discount.shape[0],
                                           rfm_discount[rfm_discount['Churn Risk'] == "At risk"].shape[0]/rfm_discount.shape[0],
                                           rfm_discount[rfm_discount['Churn Risk'] == "Churned"].shape[0]/rfm_discount.shape[0],
                                           rfm_discount.shape[0]/customers.shape[0]]

churn_segments.head(10)

<ipython-input-41-70e0616cf76a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers['RFM'] = customers['RFM'].astype(str)
<ipython-input-41-70e0616cf76a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers['Churn Risk'] = customers['RFM'].str[0].apply(lambda x: "Active" if x == "3" else ("At risk" if x == "2" else "Churned"))
<ipython-input-41-70e0616cf76a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,Segment,Active,At risk,Churned,Size
0,Urgency,0.386534,0.299252,0.314214,0.081454
1,Impulse,0.654862,0.211355,0.133783,0.361365
2,Indecisive,0.553772,0.216693,0.229535,0.126549
3,Category,0.465517,0.290640,0.243842,0.082470
4,Engagement,0.604597,0.219642,0.175760,0.874873
5,High Value Frequency,0.893841,0.067261,0.038898,0.250660
6,High Value Monetary,0.891145,0.064175,0.044679,0.250051
7,Discount,0.642775,0.201445,0.155780,0.702823


In [43]:
rfm_total = rfm(purchases, customers)

a = [rfm_total[rfm_total['Churn Risk'] == "Active"].shape[0]/rfm_total.shape[0], rfm_total[rfm_total['Churn Risk'] == "At risk"].shape[0]/rfm_total.shape[0], rfm_total[rfm_total['Churn Risk'] == "Churned"].shape[0]/rfm_total.shape[0]]
print(a)

[0.6049157018078407, 0.2208003250050782, 0.17428397318708105]


#Customer Lifetime Value

In [44]:
print(customers.columns)

Index(['Survey ResponseID', 'Prime', 'Free Trial', 'Email CTC',
       'Display Ad CTC', 'Video Ad CTC', 'Search Engine Ad CTC',
       'Social Media Ad CTC', 'Average CTC', 'Express Shipping',
       'Cart Abandonment Rate', 'Check Review', 'Refund rate', 'Time in Cart',
       'Time on Page', 'RFM', 'Pct Discount', 'Category', 'Cat_Urgency',
       'Cat_Impulse', 'Cat_Indecisive', 'Cat_Category', 'Cat_Discount',
       'Cat_Engagement', 'Cat_High_Value_F', 'Cat_High_Value_M', 'Churn Risk'],
      dtype='object')
